In [1]:
from source_code import evaluation
import numpy as np
from source_code.evaluation import *
from source_code.comparison import *
from source_code import models

import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, normalized_mutual_info_score, accuracy_score
from sklearn.preprocessing import Normalizer
import warnings
warnings.filterwarnings("ignore")

np.random.seed(42)

In [2]:
def clustering_error(y_true, y_pred):
    return 100*(1 - accuracy_score(y_true, y_pred))

In [3]:
number_list = list()
for i in range(1, 251):
    if i not in [213] :
        number_list.append(f"{i:03d}")

import random

def load_pie_dataset(n_classes):
    id_list = random.sample(number_list, n_classes)
    imgs = []
    for image_idx in id_list:
        for j in [0, 1]:
            for i in range(10):
                filename = f'small_PIE_dataset/{image_idx}/{image_idx}_01_01_051_{j}{i}_crop_128.png'
                img = load_img(filename)
                imgs.append(img)
    X = np.array(imgs)/255
    return X

In [4]:
n_classes = 3
X_data = load_pie_dataset(n_classes=n_classes)
X = X_data.reshape(X_data.shape[0], -1)
scaler = Normalizer()
X = scaler.fit_transform(X)

In [5]:
model = models.RobustPCA()
L, S = model.fit(X)

U, S, Q = np.linalg.svd(L, full_matrices=False)
labels_true = np.arange(n_classes).repeat(20)
best_err = 100

for _ in range(100):
    kmeans = KMeans(n_clusters=n_classes).fit(Q)

    pred_clusters = kmeans.labels_
    err = clustering_error(labels_true, pred_clusters)
    
    if err < best_err:
        best_err = err
        best_labels = pred_clusters
        
print(f'Best error: {best_err:.2f}%')

Best error: 58.33%


In [7]:
labels_true = np.arange(n_classes).repeat(20)
best_err = 100

for _ in range(10):
    kmeans = KMeans(n_clusters=n_classes).fit(X)

    pred_clusters = kmeans.labels_
    err = clustering_error(labels_true, pred_clusters)
    
    if err < best_err:
        best_err = err
        best_labels = pred_clusters
        
print(f'Best error: {best_err:.2f}%')

Best error: 45.00%
